In [5]:
import json
from transformers import pipeline

# Load summarization model (on CPU)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", framework="pt", device=-1)

# Load Reddit data
with open("reddit_results.json", "r", encoding="utf-8") as f:
    reddit_data = json.load(f)

# Extract all comments
all_comments = " ".join([" ".join(post["comments"]) for post in reddit_data])

# Function to split text into smaller chunks
def chunk_text(text, chunk_size=1024):
    return [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]

# Break text into smaller chunks
comment_chunks = chunk_text(all_comments, chunk_size=1024)

# Summarize each chunk separately
summaries = [summarizer(chunk, max_length=150, min_length=50, do_sample=False)[0]['summary_text'] for chunk in comment_chunks]

# Combine summaries into final summary
final_summary = " ".join(summaries)

print("\n🔹 **Overall Summary of the Reddit Discussions:**")
print(final_summary)


Device set to use cpu


KeyboardInterrupt: 